In [1]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
from transformers import pipeline
import torch

In [ ]:
# check for GPU access
print("GPU available: ", torch.cuda.is_available())
print("Current device: ", torch.cuda.current_device())
print("Device name: ", torch.cuda.get_device_name(0))

In [2]:


# Step 1: Load and Preprocess the Dataset
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df['text'] = df.apply(lambda row: f"{row['title']} {row['ingredients']} {row['directions']}", axis=1)
    return df['text'].tolist()

# Load and preprocess data
data_file = 'RecipeNLG_dataset_small.csv'
texts = load_and_preprocess_data(data_file)

# Save preprocessed texts to a file for TextDataset usage
with open("recipes.txt", "w") as f:
    for text in texts:
        f.write(text + "\n")

# Step 2: Set Up the GPT-2 Model for Fine-Tuning
model_name = "gpt2"

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

c:\Users\Mike Duran\OneDrive - Actimage GmbH\Desktop\simple-local-rag\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# Create TextDataset
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


In [5]:
# Load the dataset
dataset = load_dataset("recipes.txt", tokenizer)

c:\Users\Mike Duran\OneDrive - Actimage GmbH\Desktop\simple-local-rag\.venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [6]:
!pip install accelerate -U


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Step 3: Train the Fine-Tuned Model
training_args = TrainingArguments(
    output_dir="./gpt2-recipe-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

c:\Users\Mike Duran\OneDrive - Actimage GmbH\Desktop\simple-local-rag\.venv\Lib\site-packages\accelerate\accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [9]:
# Train the model
trainer.train()

  0%|          | 0/1602 [00:00<?, ?it/s]

In [ ]:
# Step 4: Save the Fine-Tuned Model
model.save_pretrained("./gpt2-recipe-finetuned")
tokenizer.save_pretrained("./gpt2-recipe-finetuned")

In [ ]:
# Step 5: Create the generate_response Method
def generate_response(prompt, context, model, tokenizer, max_length=100):
    input_text = f"{context} {prompt}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)
    
    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example usage of generate_response
finetuned_model = GPT2LMHeadModel.from_pretrained("./gpt2-recipe-finetuned")
finetuned_tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-recipe-finetuned")

prompt = "How do I make a chocolate cake?"
context = "Here is a recipe for a delicious chocolate cake."

response = generate_response(prompt, context, finetuned_model, finetuned_tokenizer)
print(response)
